In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 60

## Inspecting and cleaning data

In [3]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
England=final[final['country_id']=='England Premier League'].sort(columns='Date')
Belgium=final[final['country_id']=='Belgium Jupiler League'].sort(columns='Date')
France=final[final['country_id']=='France Ligue 1'].sort(columns='Date')
Germany=final[final['country_id']=='Germany 1. Bundesliga'].sort(columns='Date')
Italy=final[final['country_id']=='Italy Serie A'].sort(columns='Date')
Netherlands=final[final['country_id']=='Netherlands Eredivisie'].sort(columns='Date')
Poland=final[final['country_id']=='Poland Ekstraklasa'].sort(columns='Date')
Portugal=final[final['country_id']=='Portugal Liga ZON Sagres'].sort(columns='Date')
Scotland=final[final['country_id']=='Scotland Premier League'].sort(columns='Date')
Spain=final[final['country_id']=='Spain LIGA BBVA'].sort(columns='Date')
Swiss=final[final['country_id']=='Switzerland Super League'].sort(columns='Date')
England.reset_index(inplace=True)
Belgium.reset_index(inplace=True)
France.reset_index(inplace=True)
Germany.reset_index(inplace=True)
Italy.reset_index(inplace=True)
Netherlands.reset_index(inplace=True)
Poland.reset_index(inplace=True)
Portugal.reset_index(inplace=True)
Scotland.reset_index(inplace=True)
Spain.reset_index(inplace=True)
Swiss.reset_index(inplace=True)
England.drop(['index'],axis=1,inplace=True)
Belgium.drop(['index'],axis=1,inplace=True)
France.drop(['index'],axis=1,inplace=True)
Germany.drop(['index'],axis=1,inplace=True)
Italy.drop(['index'],axis=1,inplace=True)
Netherlands.drop(['index'],axis=1,inplace=True)
Poland.drop(['index'],axis=1,inplace=True)
Portugal.drop(['index'],axis=1,inplace=True)
Scotland.drop(['index'],axis=1,inplace=True)
Spain.drop(['index'],axis=1,inplace=True)
Swiss.drop(['index'],axis=1,inplace=True)

## Working with Italy

In [9]:
def cut_out_n_m_matches(df,n,m):
    return df[(df['H_ROUND']>n) & (df['H_ROUND']<m)]
    

In [10]:
Italy=cut_out_n_m_matches(Italy,4,35)

## Splitting Data 

In [11]:
col=[46,47]
pred='FTR'

In [19]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [20]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
    
    
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]

    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]

    return X_train, Y_train, X_test, Y_test
    

In [ ]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
    tmp[method_name]=lis 
    
    return tmp

## SVM_LINEAR

In [23]:
#SVM_LINEAR_whole_season_by_one(Italy,'2015/2016',[46,47], 'FTR', 'svm_linear')

## SVM_RBF

In [ ]:
def SVM_RBF_whole_season_by_one (BIG, predseason, col_train, col_pred, method):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        
        svm_linear = svm.SVC(kernel='linear')
        svm_linear.fit(X_train,Y_train)
        lis.append(float(svm_linear.score(X_test, Y_test)))
        
    tmp[method]=lis 
    
    return tmp
    

In [26]:
def predict_whole_season_with_one (BIG, predseason, col_train, col_pred):
        
    tmp=pd.DataFrame(columns=['svm_linear','svm_rbf', 'cart','begging', 'adaBoo','gradient',' randomF'])
    k=0
    
    train_index=BIG[BIG.season!=predseason].index
    test_index=BIG[BIG.season==predseason].index
    
    for i in range(5,9):
        print(i)
    #BIG[BIG.season==predseason].H_ROUND.unique():
    
        lis=[]
            
        X_train=BIG.loc[train_index].iloc[:,col_train]
        Y_train=BIG.loc[train_index][col_pred]
    
        X_test=BIG.loc[test_index]
        Y_test=BIG.loc[test_index]
        
        X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
        Y_test=Y_test[Y_test.H_ROUND==i][col_pred]
        
        
        svm_linear = svm.SVC(kernel='linear')
        svm_linear.fit(X_train,Y_train)
        lis.append(float(svm_linear.score(X_test, Y_test)))
        
        svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
        svm_rbf.fit(X_train,Y_train)
        lis.append(float(svm_rbf.score(X_test, Y_test)))
        
        dtc = DecisionTreeClassifier()
        dtc.fit(X_train, Y_train)
        lis.append(float(dtc.score(X_test,Y_test)))
        
        bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
        bagb.fit(X_train,Y_train)
        lis.append(float(bagb.score(X_test,Y_test)))
        
        adab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
        adab.fit(X_train,Y_train)
        lis.append(float(adab.score(X_test,Y_test)))
        
        
        gradb = GradientBoostingClassifier(max_depth=10, n_estimators=100)
        gradb.fit(X_train,Y_train)
        lis.append(float(gradb.score(X_test,Y_test)))
        
        rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
        rdf.fit(X_train,Y_train)
        lis.append(float(rdf.score(X_test,Y_test)))
    
        tmp.loc[k]=lis
        k=k+1
        
        

        train_index=list(train_index)+list(X_test.index)
        
        
        
        
        
    return tmp
    

In [425]:
tmp1=predict_whole_season_by_one(Italy,'2015/2016',[46,47], 'FTR')

In [ ]:
tmp1.mean()

In [421]:
tmp2=predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR')

5
2101
10
6
2111
10
7
2121
10
8
2131
10


In [413]:
tmp2

,svm_linear,svm_rbf,cart,begging,adaBoo,gradient,randomF
0,0.5,0.5,0.4,0.4,0.3,0.4,0.4
1,0.6,0.5,0.3,0.3,0.3,0.2,0.4
2,0.3,0.5,0.3,0.5,0.5,0.4,0.5
3,0.5,0.4,0.6,0.5,0.6,0.4,0.6
4,0.6,0.3,0.3,0.3,0.6,0.3,0.6
5,0.8,0.7,0.6,0.8,0.8,0.6,0.6
6,0.3,0.1,0.3,0.2,0.4,0.2,0.1
7,0.4,0.5,0.5,0.4,0.6,0.6,0.7
8,0.5,0.6,0.6,0.6,0.7,0.6,0.6
9,0.5,0.6,0.3,0.6,0.5,0.5,0.6


# SVM

In [27]:
scoreslinear=[]
scoresrbf=[]

X=Italy.iloc[:,[46,47]]
Y=Italy['FTR']


for i in range (0,1):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20)
    
    svm_linear = svm.SVC(kernel='linear')
    svm_linear.fit(X_train,Y_train)
    scoreslinear.append(svm_linear.score(X_test, Y_test))
    
    svm_rbf = svm.SVC(kernel='rbf')
    svm_rbf.fit(X_train,Y_train)
    scoresrbf.append(svm_rbf.score(X_test, Y_test))

In [28]:
svm_linear = svm.SVC(kernel='linear')
a=svm_linear.fit(X_train,Y_train)
svm_linear.score(X_test, Y_test)

0.48856548856548859

In [32]:
a=svm_linear.decision_function(X_train)

In [44]:
a[-1]

array([ 0.85030201,  0.32892017, -1.00002676])

In [50]:
svm_linear.predict(X_train)=='H'

array([ True,  True,  True, ...,  True, False, False], dtype=bool)

In [383]:
sum(scoresrbf)/len(scoresrbf)


0.49322245322245317

In [384]:
sum(scoreslinear)/len(scoreslinear)

0.48700623700623724

## CART

In [385]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, random_state=4)
ts = Y_test.shape[0]

In [289]:
X_train=Italy.loc[:2946,:].iloc[:,[46,47]]
Y_train=Italy.loc[:2946,:]['FTR']

a=Italy[Italy.season=='2015/2016'].H_ROUND==32
X_test=Italy[Italy.season=='2015/2016'][a].iloc[:,[46,47]]
Y_test=Italy[Italy.season=='2015/2016'][a]['FTR']

In [386]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, Y_train)
dtc.score(X_test,Y_test)



0.38141470180305131

In [291]:
#Y_test[dtc_verror.astype(bool)]

# Bagging Classifier 

In [387]:
bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
#adab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=20,learning_rate=1.5,algorithm="SAMME")
bagb.fit(X_train,Y_train)
bagb.score(X_test,Y_test)

0.41192787794729541

# AdaBoost Classifier

In [388]:
adab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
#adab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=20,learning_rate=1.5,algorithm="SAMME")
adab.fit(X_train,Y_train)
adab.score(X_test,Y_test)

0.39667128987517336

# Gradient Boosting Classifier


In [390]:
gradb = GradientBoostingClassifier(max_depth=10, n_estimators=200)
gradb.fit(X_train,Y_train)
gradb.score(X_test,Y_test)

0.42441054091539526

# Random Forest

In [391]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=20)

rdf.fit(X_train,Y_train)
rdf.score(X_test,Y_test)


0.44798890429958393

In [ ]:
gradb.fit(X_train,Y_train)
gradb.score(X_test,Y_test)
rdf.fit(X_train,Y_train)
rdf.score(X_test,Y_test)

In [ ]:
X_train=Italy.loc[:2946,:].iloc[:,[46,47]]
Y_train=Italy.loc[:2946,:]['FTR']

a=Italy[Italy.season=='2015/2016'].H_ROUND==33
X_test=Italy[Italy.season=='2015/2016'][a].iloc[:,[46,47]]
Y_test=Italy[Italy.season=='2015/2016'][a]['FTR']